# Swaption Fundamentals

This notebook demonstrates how to access and use the **Swaption** functionalities within the **LSEG Financial Analytics SDK**. 

Swaption instruments are interest rate derivatives that provide protection against adverse interest rate movements. The following swaption types are supported:
- Payer Swaption: Right to enter a swap as the fixed-rate payer (protection against rising rates)
- Receiver Swaption: Right to enter a swap as the fixed-rate receiver (protection against falling rates)

With following exercise styles:
- European Style: Exercisable at expiration date
- American Style: Exercisable at any date before expiration
- Bermudan Style: Exercisable on multiple specified dates

**You will be able to:**
- Define Swaption instruments with basic parameters
- Configure pricing parameters
- Evaluate Swaption Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.swaption` - for Swaption instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import swaption  as sw

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Swaption instrument you need to follow a structured 3-step process:

1. **Swaption Definition** - Specify basic swaption parameters (strike, expiry, underlying swap details)
2. **Swaption Instrument Definition** - Create the instrument object for pricing
3. **Pricing Preferences** - Configure pricing parameters and market data settings, optional



## 

In [2]:
print("Step 1: Creating Swaption Definition...")

# Define a European payer swaption (protection against rising rates)
swaption_definition = sw.SwaptionDefinition(
    # Core Swaption Parameters
    buy_sell=sw.IPABuySellEnum.BUY,                            # Buy swaption protection. Options: BUY, SELL
    swaption_type=sw.SwaptionTypeEnum.PAYER,                   # Payer swaption (pay fixed, receive floating). Options: PAYER, RECEIVER
    exercise_style=sw.IPAExerciseStyleEnum.EURO,               # European exercise style. Options: EURO, AMER, BERM
    
    # Option Timing
    start_date=dt.datetime(2025, 7, 18),                       # Option start date (defaults to valuation_date)
    end_date=dt.datetime(2026, 1, 15),                         # Option expiry date
    # Alternative: tenor="6M",                                 # Or use tenor instead of end_date
    
    # Financial Terms
    notional_amount=10_000_000,                                # $10M notional (default: 1,000,000)
    strike_percent=3.50,                                       # 3.50% strike rate (fixed rate of underlying swap)
    
    # Settlement Configuration
    settlement_type=sw.SettlementTypeEnum.PHYSICAL,           # Physical settlement. Options: PHYSICAL, CASH, CCP
    premium_settlement_type=sw.PremiumSettlementTypeEnum.SPOT, # Premium settlement timing. Options: SPOT, FORWARD, SCHEDULE
    delivery_date=dt.datetime(2026, 1, 17),                   # Settlement date (optional, defaults to end_date)
    
    # Optional Parameters
    instrument_tag="USD_5Y_PAYER_SWAPTION",                   # User-defined identifier (max 40 chars)
    spread_vs_atm_in_bp=0.0,                                  # Spread vs ATM in basis points (for pre-trade scenarios)
    
    # Underlying swap details (using SwapDefinition). Detailed swap definition setups are covered in IR Swap SDK
    underlying_definition=sw.SwapDefinition(
        template="OIS_SOFR",                                   # Standard USD SOFR swap template
        start_date=dt.datetime(2026, 1, 17),                   # Swap start (T+2 from option expiry)
        end_date=dt.datetime(2031, 1, 17),                     # 5Y swap tenor in date format
        instrument_tag="USD_5Y_SOFR_Underlying_Swap"           # Tag for underlying swap
    ),
    
)

# Display comprehensive information about the created swaption
print(f"   Swaption Type: {swaption_definition.swaption_type}")
print(f"   Exercise Style: {swaption_definition.exercise_style}")
print(f"   Buy/Sell: {swaption_definition.buy_sell}")
print(f"   Notional Amount: ${swaption_definition.notional_amount:,.0f}")
print(f"   Strike Rate: {swaption_definition.strike_percent}%")
print(f"   Option Start: {swaption_definition.start_date.strftime('%Y-%m-%d')}")
print(f"   Option Expiry: {swaption_definition.end_date.strftime('%Y-%m-%d')}")
print(f"   Settlement Type: {swaption_definition.settlement_type}")
print(f"   Premium Settlement: {swaption_definition.premium_settlement_type}")
print(f"   Underlying Template: {swaption_definition.underlying_definition.template}")
print(f"   Instrument Tag: {swaption_definition.instrument_tag}")
print()

print("Step 2: Creating Instrument Object...")
swaption_instrument = sw.SwaptionDefinitionInstrument(definition=swaption_definition)
print("   Instrument container created for pricing")
print()

print("Step 3: Configuring Pricing Parameters...")
pricing_params = sw.SwaptionPricingParameters(
    valuation_date=dt.datetime(2025, 7, 18),                   # Pricing date
    price_side=sw.PriceSideEnum.MID                            # Use mid-market prices. Options: BID, ASK, MID, LAST
)
print(f"   Valuation Date: {pricing_params.valuation_date.strftime('%Y-%m-%d')}")
print(f"   Price Side: {pricing_params.price_side}")

Step 1: Creating Swaption Definition...
   Swaption Type: SwaptionTypeEnum.PAYER
   Exercise Style: IPAExerciseStyleEnum.EURO
   Buy/Sell: IPABuySellEnum.BUY
   Notional Amount: $10,000,000
   Strike Rate: 3.5%
   Option Start: 2025-07-18
   Option Expiry: 2026-01-15
   Settlement Type: SettlementTypeEnum.PHYSICAL
   Premium Settlement: PremiumSettlementTypeEnum.SPOT
   Underlying Template: OIS_SOFR
   Instrument Tag: USD_5Y_PAYER_SWAPTION

Step 2: Creating Instrument Object...
   Instrument container created for pricing

Step 3: Configuring Pricing Parameters...
   Valuation Date: 2025-07-18
   Price Side: PriceSideEnum.MID


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instrument definitions for batch processing

try:
    response = sw.price(
        definitions=[swaption_instrument],
        pricing_preferences=pricing_params,
        fields="MarketValueInDealCcy, premiumPercent, premiumBp, deltaPercent, gammaPercent, vegaPercent, thetaPercent" # optional, used to build the tabular data explained below
    )

    # Display response structure information
    analytics_data = response['data']['analytics'][0]
    if analytics_data['error'] == {}:
        print("   Calculation successful!")
        valuation_data = analytics_data.get('valuation', {})
    else:
        print(f"   Pricing error: {analytics_data['error']}")
        
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise


   Calculation successful!


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike rate, expiry dates, notional, underlying swap structure), using the SwaptionDefinition that we created.

 - **pricingPreferences**: Valuation date, price side (BID/ASK/MID/LAST), currency settings.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`)
   - **description**: Instrument summary SwaptionDefinition and default fields used in the calculation
   - **greeks**: Main risk metrics calculated on swaption instrument (`deltaPercent`, `gammaPercent`, `vegaPercent`, `thetaPercent`)
   - **valuation**: Detailed valuation of the swaption instrument with the below fields:
      - `MarketValueInDealCcy`: Present value of the swaption in deal currency
      - `PremiumPercent`: Swaption premium as percentage of notional
      - `PremiumBp`: Swaption premium in basis points
   - **error**: Root cause of any issues, if encountered

We give below the valuation details, as well as the tabular data details, that include information on both valuation and greeks .

In [4]:
# Access the valuation object
print("Swaption Valuation Results:")
print(json.dumps(valuation_data.as_dict(), indent=4))

Swaption Valuation Results:
{
    "marketValueInDealCcy": 123946.424798278,
    "premiumPercent": 1.23946424798278,
    "premiumBp": 123.946424798278
}


In [5]:
# Convert swaption analytics in tabular data to DataFrame for better visualization
print("Swaption Analytics Summary:")

columns = [item.name for item in analytics_data.tabular_data.headers]
response_data = analytics_data.tabular_data.data

response_df = pd.DataFrame([response_data], columns=columns).T
response_df[0] = pd.to_numeric(response_df[0], errors='coerce').round(2)
response_df.columns = ['Value']  # Add meaningful column name

display(response_df)

Swaption Analytics Summary:


,Value
MarketValueInDealCcy,123946.42
PremiumPercent,1.24
PremiumBp,123.95
DeltaPercent,2.33
GammaPercent,274.00
VegaPercent,1.26
ThetaPercent,-0.00
ErrorCode,NaN
ErrorMessage,NaN


## Appendix: Main Functions Reference

The following is a list of main classes, enums, and functions available after importing from  `lseg_analytics.pricing.instruments.swaption`:

**Core Classes:**
- `SwaptionDefinition` - Define swaption instrument specifications (strike, expiry, underlying swap)
- `SwaptionDefinitionInstrument` - Instrument container for pricing operations
- `BermudanSwaptionDefinition` - Bermudan-style swaption with multiple exercise dates
- `CancellableDefinition` - Cancellable swap (*i.e.* an interest rate swap + a receiver/payer swaption) definition
- `SwapDefinition` - Underlying swap structure definition
- `SwapLegDefinition` - Individual swap leg specifications (fixed/floating)
- `SwaptionPricingParameters` - Configure pricing parameters and market data settings

**Response Classes:**
- `SwaptionCalculationResponse` - Main response container
- `SwaptionAnalyticsResponseData` - Detailed analytics and calculations
- `SwaptionAnalyticsResponseWithError` - Analytics response with error
- `SwaptionValuation` - Individual instrument valuation results
- `SwaptionDescription` - Instrument description
- `SwaptionPricingAnalysis` - Swaption pricing analysis
- `SwaptionGreeks` - Risk sensitivities (delta, gamma, vega, theta)
- `SwaptionCashflows` - Detailed cashflow schedules

**Swaption Configuration Enums:**
- `SwaptionTypeEnum` - Swaption type: PAYER, RECEIVER
- `IPAExerciseStyleEnum` - Exercise style: EURO, AMER, BERM
- `OptionOwnerEnum` - Option ownership direction: COUNTERPARTY, SELF_ENTITY
- `SettlementTypeEnum` - Settlement method: PHYSICAL, CASH, CCP
- `PremiumSettlementTypeEnum` - Premium payment timing: SPOT, FORWARD, SCHEDULE
- `ExerciseScheduleTypeEnum` - Source of the exercise dates of Bermudan options: FIXED_LEG, FLOAT_LEG, USER_DEFINED

**Swap Configuration Enums:**
- `IPADirectionEnum` - Leg direction: PAID, RECEIVED
- `InterestPaymentFrequencyEnum` - Payment frequencies: ANNUAL, SEMI_ANNUAL, QUARTERLY, MONTHLY, etc.
- `IndexResetFrequencyEnum` - Reset frequencies: EVERYDAY, MONTHLY, QUARTERLY, etc.
- `InterestCalculationMethodEnum` - Day count conventions: DCB_ACTUAL_360, DCB_30_360, etc.
- `PaymentBusinessDayConventionEnum` - Business day adjustment rules: MODIFIED_FOLLOWING, NEXT_BUSINESS_DAY, etc.

**Market Data Enums:**
- `PriceSideEnum` - Market price sides: BID, ASK, MID, LAST
- `IndexPriceSideEnum` - Index price side selection: BID, ASK, MID, LAST
- `IPAIndexObservationMethodEnum` - Index observation methods: LOOKBACK, MIXED, PERIOD_SHIFT

**Trading Enums:**
- `IPABuySellEnum` - Buy/Sell direction: BUY, SELL
- `TypeEnum` - General type classifications

**Main Functions:**
- `price()` - Execute swaption pricing calculations
- `functions_all()` - List all available functions in the module